In [ ]:
!pip install guardrails-ai
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
import os
import guardrails as gd

In [ ]:
GPT_API_KEY = "sk-"

os.environ["OPENAI_API_KEY"] = GPT_API_KEY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DIR       = "./"
TEXT_FILE = "onepiece.txt"

#RAIL_FILE = "test.rail"

In [ ]:
with open(DIR + TEXT_FILE) as f:
  text = f.read()

In [ ]:
rail_str = """
<rail version="0.1">

<output>

    <object name = "list_tags_object">
      <list name = "tags" description = "次の文章の特徴を表す10個のタグを日本語でつけるとしたらどんなタグ?">
          <object>
              <integer name = "index" format = "1-indexed" />
              <string  name = "tag"   format="one-word"  on-fail-one-word="reask"/>
          </object>
      </list>
    </object>

</output>


<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

{{document}}

@xml_prefix_prompt

{output_schema}

@json_suffix_prompt_v2_wo_none</prompt>

</rail>
"""

In [ ]:

guard = gd.Guard.from_rail_string(rail_str)

#guard = gd.Guard.from_rail(DIR + RAIL_FILE)

# Warningsが出る場合があるけれども無視してOK

In [ ]:
# Wrap the LLM API call with Guard

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params = {"document": text},
    engine        = "text-davinci-003",
    max_tokens    = 1024,
    temperature   = 0.0
)

In [ ]:
print(validated_response)

{'list_tags_object': {'tags': [{'index': 1, 'tag': '冒険'}, {'index': 2, 'tag': '別れ'}, {'index': 3, 'tag': '修行'}, {'index': 4, 'tag': '海賊'}, {'index': 5, 'tag': 'コビー'}, {'index': 6, 'tag': 'アルビダ'}, {'index': 7, 'tag': 'ゾロ'}, {'index': 8, 'tag': 'ナミ'}, {'index': 9, 'tag': 'ウソップ'}, {'index': 10, 'tag': 'サンジ'}]}}


In [ ]:
# JSON形式を見やすくする

import pprint
pprint.pprint(validated_response)

{'list_tags_object': {'tags': [{'index': 1, 'tag': '冒険'},
                               {'index': 2, 'tag': '別れ'},
                               {'index': 3, 'tag': '修行'},
                               {'index': 4, 'tag': '海賊'},
                               {'index': 5, 'tag': 'コビー'},
                               {'index': 6, 'tag': 'アルビダ'},
                               {'index': 7, 'tag': 'ゾロ'},
                               {'index': 8, 'tag': 'ナミ'},
                               {'index': 9, 'tag': 'ウソップ'},
                               {'index': 10, 'tag': 'サンジ'}]}}
